In [1]:
## Import Packages
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup

import os
import re
import pandas as pd
import csv

In [8]:
## Set Browser to Google Chrome
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs", prefs)


In [9]:
## Set Facebook event group 
driver = webdriver.Chrome("D:\Python>", chrome_options=chrome_options)
url = 'https://www.facebook.com/groups/##your_group_name_##/events'

C:\Users\mader\AppData\Local\Temp\ipykernel_39036\3314076339.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:\Python>", chrome_options=chrome_options)
C:\Users\mader\AppData\Local\Temp\ipykernel_39036\3314076339.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("D:\Python>", chrome_options=chrome_options)


In [10]:
## Open facebook group website page
driver.get(url)

In [11]:
## Define a pause for loading a page
def wait_for_page_load():
    timer= 10
    start_time = time.time()
    page_state = None
    while page_state != 'complete':
        time.sleep(0.5)
        page_state = driver.execute_script('return document.readyState;')
        if time.time() - start_time > timer:
            raise Exception('Timeout :(')

In [12]:
## Login to facebook account
username = "your@email.here"
password = "Your_Password_Here"
driver.implicitly_wait(2)

login = driver.find_element(By.ID, 'email')
login.send_keys(username)

pw = driver.find_element(By.ID, 'pass')
pw.send_keys(password)

driver.implicitly_wait(2)

LI = driver.find_element(By.ID, 'loginbutton')

LI.click()

time.sleep(5)

In [ ]:
## infinite scroll "see more" to show all href links

while(driver.find_element(By.CSS_SELECTOR, "[aria-label='See more']")):
    driver.find_element(By.CSS_SELECTOR, "[aria-label='See more']").click()
    time.sleep(7)

In [ ]:
## recall all links (if necessary), otherwise skip to next step

anchors = driver.find_elements(By.XPATH,'//a[contains(@href,"event_action_history")]')
for a in anchors:
    print([a.get_attribute("href")])

In [13]:
## define links that we need to move to a csv
anchors = driver.find_elements(By.XPATH,'//a[contains(@href,"event_action_history")]')

In [14]:
## count number of links
count = len(anchors)
print(count)

6


In [18]:
## record anchors into csv 

with open('href_links.csv', 'w', newline='' ) as rw:
    hwriter = csv.writer(rw)
    for a in anchors:
        hwriter.writerow([a.get_attribute("href")])

In [15]:
## iterate through the list in the csv and record data into separate csv. the xpaths need to be checked/updated if necessary

header = ['event_name', 'event_creator', 'event_date', 'event_duration', 'event_location', 'going']
with open('scraped_data.csv', 'w') as t:
    writer = csv.writer(t)
    writer.writerow(header)

    hrefs = []
    with open('href_links.csv', 'r') as r:
        reader = csv.reader(r)
        for row in reader:
            links = row[0]
            hrefs.append(links)
    
    for link in hrefs:
        url_html = requests.get(link)
        
        hrefs = hrefs[0:x] ## x = the total amount of links can be found either in the csv or from print(count) function in row 14. 

        hrefs2 = {}

        for href in hrefs:
            hrefs2[href] = []
            driver.get(href)
            wait_for_page_load()

            try:
                event_name = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[2]/div/div[1]/div/div/div[2]/h2/span/span/span").text
            except:
                event_name = "No Data"
    
           
            try:
                event_creator = driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[2]/div/div/div/div/div[1]/div/div/div/div[3]/div/div/div[2]/div/div/span/strong[1]/a').text
            except:
                event_creator = "No Data"

            try: 
                event_location = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[2]/div/div/div/div/div[1]/div/div/div/div[4]').text
            except:
                event_creator = "No Data"

            try:
                event_duration = driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[2]/div/div/div/div/div[1]/div/div/div/div[5]/div/div/div[2]/div/div/span').text        
            except:
                event_duration = "No Data"

            try:
                event_date = driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[2]/div/div[1]/div/div/div[1]/span').text
            except:
                event_data = "No Data"
   
    
            guest_list = driver.find_element(By.XPATH,'//span[contains(@class, "x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x676frb x1nxh6w3 x1sibtaa x1s688f xi81zsa x2b8uid")]')
            guest_list.click()
            time.sleep(50)
            going = driver.find_elements(By.XPATH,'//span[contains(@class, "x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x6prxxf xvq8zen xo1l8bm xzsf02u")]')


        
            going_text = [g.text for g in going]

            writer.writerow([event_name, event_creator, event_date, event_duration, event_location, going_text])
